Background reading: http://mlwave.com/kaggle-ensembling-guide/

In [1]:
import pandas as pd

In [2]:
# Load submission files

"""
gb_model: no feature engineering, 7 features, test accuracy = 0.83, AUC = 0.859
log_model: feature engineering, 4 features, test accuracy = 0.843, AUC = 0.872 (performed better than
random forests in public submissions)
forest_model2: feature engineering, 7 features, test accuracy = 0.843, AUC = 0.859

"""

log_model = pd.read_csv('submission_logistic.csv')
forest_model = pd.read_csv('submission_forest.csv')
gb_model = pd.read_csv('submission_gradientboost.csv')
forest_model2 = pd.read_csv('submission_forest2.csv')

In [3]:
log_model.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [4]:
forest_model.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0


In [5]:
gb_model.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [ ]:
# question is what weight to give logistic regression 
# and how to determine whether a female in 3rd class survived

In [24]:
w = 2.1
vote_DF = pd.DataFrame({'Forest1': forest_model.Survived, 'Forest2': forest_model2.Survived, \
                       'GBBoost': gb_model.Survived, 'Logistic': log_model.Survived * w})

# Model correlations

In [7]:
vote_DF.corr()

,Forest1,Forest2,GBBoost,Logistic
Forest1,1.000000,0.669349,0.792629,0.635629
Forest2,0.669349,1.000000,0.717115,0.685898
GBBoost,0.792629,0.717115,1.000000,0.675364
Logistic,0.635629,0.685898,0.675364,1.000000


# Majority vote

In [25]:
vote_DF[100:150]

,Forest1,Forest2,GBBoost,Logistic
100,1,1,1,2.1
101,0,0,0,0.0
102,0,0,0,0.0
103,0,0,0,0.0
104,1,1,1,2.1
105,0,0,0,0.0
106,0,0,0,0.0
107,0,0,0,0.0
108,0,0,0,0.0
109,0,0,0,0.0


In [31]:
log_model.iloc[141]

PassengerId    1033
Survived          1
Name: 141, dtype: int64

In [10]:
print(sum(forest_model.Survived), sum(forest_model2.Survived), sum(gb_model.Survived), sum(log_model.Survived))

154 150 144 164


In [26]:
vote_series = []
for i in range(len(vote_DF)):
    row_total = sum(vote_DF.iloc[i])
    if row_total <= 2:
        vote_series.append(0)
    else:
        vote_series.append(1)

In [27]:
sum(vote_series)

170

In [13]:
len(vote_series)

418

In [14]:
rowSumList = [sum(vote_DF.iloc[i]) for i in range(len(vote_DF))]

In [23]:
# look at models and see which ones are most frequently in 'alignment' when 3 are in agreement
count = 0
logCount = 0
gbCount = 0
forest1Count = 0
forest2Count = 0
for i in range(len(vote_DF)):
    if rowSumList[i] == 3:
        count += 1
        if vote_DF.Logistic[i] == 1 * w:
            logCount += 1
        if vote_DF.GBBoost[i] == 1:
            gbCount += 1
        if vote_DF.Forest1[i] == 1:
            forest1Count += 1
        if vote_DF.Forest2[i] == 1:
            forest2Count += 1
print('Logistic', count, logCount)
print('GBBoost', count, gbCount)
print('Forest1', count, forest1Count)
print('Forest2', count, forest2Count)

Logistic 6 0
GBBoost 6 6
Forest1 6 6
Forest2 6 6


In [76]:
# look at models and see which ones are most frequently suggesting survival when other 3 disagree
count = 0
logCount = 0
gbCount = 0
forest1Count = 0
forest2Count = 0
for i in range(len(voteSuc)):
    if rowSumList[i] == 1:
        count += 1
        if vote_DF.Logistic[i] == 1 * w:
            logCount += 1
        if vote_DF.GBBoost[i] == 1:
            gbCount += 1
        if vote_DF.Forest1[i] == 1 * w:
            forest1Count += 1
        if vote_DF.Forest2[i] == 1:
            forest2Count += 1
print('Logistic', count, logCount)
print('GBBoost', count, gbCount)
print('Forest1', count, forest1Count)
print('Forest2', count, forest2Count)

Logistic 58 19
GBBoost 58 3
Forest1 58 17
Forest2 58 19


# New submission

In [66]:
# Create a new dataframe with only the columns Kaggle wants from the dataset.
submission = pd.DataFrame({
        "PassengerId": log_model["PassengerId"],
        "Survived": vote_series
    })

# Any files you save will be available in the output tab below
submission.to_csv('submission.csv', index=False)